In [2]:
from rocketcea.cea_obj_w_units import CEA_Obj
# from rocketcea.cea_obj import add_new_fuel
# from pyfluids import Fluid, FluidsList, Input
# import numpy as np

# class combustion_results:
#     def __init__(self, tc, cp):
#         self.tc = tc
#         self.cp = cp
    
#     def pretty_print(self):
#         print(f"Combustion Results:")
#         print(f"  {'Combustion Temperature (Tc)':<30} {self.tc:<10.4g} K")
#         print(f"  {'Combustion Heat Capacity (Cp)':<30} {self.cp:<10.4g} J/kg/K")

class GasGenerator:
    def __init__(self, ox: str, fuel: str):
        self.ox = ox
        self.fuel = fuel
        self.cea = CEA_Obj(
            oxName = self.ox,
            fuelName = self.fuel,
            isp_units='sec',
            cstar_units = 'm/s',
            pressure_units='Bar',
            temperature_units='K',
            sonic_velocity_units='m/s',
            enthalpy_units='J/g',
            density_units='kg/m^3',
            specific_heat_units='J/kg-K',
            viscosity_units='centipoise', # stored value in pa-s
            thermal_cond_units='W/cm-degC', # stored value in W/m-K
            # fac_CR=self.cr,
            make_debug_prints=False)
    def combustion_sim(self, OF, pc):
        Tg_c = self.cea.get_Tcomb(Pc=pc, MR=OF)
        [mw_c, gam_c] = self.cea.get_Chamber_MolWt_gamma(Pc=pc, MR=OF, eps=40)
        R_c = 8314.5 / mw_c  # J/kg-K
        Cp_c = self.cea.get_Chamber_Cp(Pc=pc, MR=OF, eps=40)
        return Tg_c, Cp_c, R_c, gam_c


In [ ]:
# To see variance of combustion properties between chamber, throat and exit. Not too much so I'll assume everything is chamber for now.
# from rocketcea.cea_obj import CEA_Obj

# ispObj = CEA_Obj( oxName='N2O', fuelName='Isopropanol')
# s = ispObj.get_full_cea_output( Pc=50, # number or list of chamber pressures
#                                 MR=3.5,   # number or list of mixture ratios
#                                 # PcOvPe=5,# number or list of Pc/Pexit
#                                 eps=4,   # number or list of supersonic area ratios
#                                 subar=5,     # number or list of subsonic area ratios
#                                 short_output=1,  # 0 or 1 to control output length
#                                 pc_units='bar', # pc_units = 'psia', 'bar', 'atm', 'mmh'
#                                 output='siunits',# output = 'calories' or 'siunits'
#                                 fac_CR=None)     # finite area combustor, contraction ratio
# pr    int( s )


 *******************************************************************************

         NASA-GLENN CHEMICAL EQUILIBRIUM PROGRAM CEA, OCTOBER 18, 2002
                   BY  BONNIE MCBRIDE AND SANFORD GORDON
      REFS: NASA RP-1311, PART I, 1994 AND NASA RP-1311, PART II, 1996

 *******************************************************************************



 reac
  fuel C3H8O-2propanol C 3 H 8 O 1    wt%=100.
  h,cal=-65133.     t(k)=298.15   rho=0.786
  oxid NitrousOxide  N 2.0 O 1.0  wt%=100.00
  h,cal= 19467.0 t(k)=298.15
  
 prob case=RocketCEA,
  rocket equilibrium   p,bar=50.000000,  subar=5.000000,  supar=4.000000,
  o/f=3.500000,
  
  
 output siunits  short  transport
 end






              THEORETICAL ROCKET PERFORMANCE ASSUMING EQUILIBRIUM

           COMPOSITION DURING EXPANSION FROM INFINITE AREA COMBUSTOR

 Pinj =   725.2 PSIA
 CASE = RocketCEA,     

             REACTANT                    WT FRACTION      ENERGY      TEMP
                                      

In [ ]:
import numpy as np

class Turbine:
    def __init__(self, 
                 P,       # W
                 RPM,     
                 d_mean_mm, # mm
                 T01,       # K
                 gamma,
                 mdot,     # kg/s
                 cp,       # J/kg/K
                 R,         # J/kg/K
                 p01_bar,    # bar
                 beta_deg,   # degrees
                 doa,       # degree of admission
                 pamb):     # ambient pressure [Pa]
        
        # Store inputs
        self.P = P
        self.RPM = RPM
        self.d_mean_mm = d_mean_mm
        self.T01 = T01
        self.gamma = gamma
        self.mdot = mdot
        self.cp = cp
        self.R = R
        self.p01_bar = p01_bar
        self.beta = np.deg2rad(beta_deg)
        self.doa = doa
        self.pamb = pamb
        
        # Derived inputs
        self.d_mean = d_mean_mm / 1000
        self.p01 = p01_bar * 1e5

        # Run calculations immediately
        self._compute()

    def _compute(self):
        # Angular velocity & blade speed
        self.w = self.RPM * 2 * np.pi / 60
        self.u = self.w * self.d_mean / 2
        
        # Useful enthalpy drop
        self.deltah_useful = self.P / self.mdot
        
        # Velocity triangles
        self.c3u = self.deltah_useful / (2 * self.u) + self.u
        self.c3 = self.c3u / np.cos(self.beta)
        self.c3m = self.c3 * np.sin(self.beta)
        
        # Outlet conditions
        self.T3 = self.T01 - 0.5 * self.c3**2 / self.cp
        self.a3 = np.sqrt(self.gamma * self.R * self.T3)
        self.M3 = self.c3 / self.a3
        
        # Pressure & density at blade inlet
        self.p3 = self.p01 / (1 + 0.5 * (self.gamma - 1) * self.M3**2)**(self.gamma/(self.gamma-1))
        self.rho_3 = self.p3 / (self.R * self.T3)
        
        # Blade height
        self.Height = self.mdot / (self.doa * self.rho_3 * self.c3m * self.d_mean)
        
        # Isentropic expansion to ambient
        self.T_amb_is = self.T01 * (self.pamb/self.p01)**((self.gamma - 1)/self.gamma)
        self.deltah_ambis = self.cp * (self.T01 - self.T_amb_is)
        
        # Efficiency
        self.eff = self.deltah_useful / self.deltah_ambis

        # Blade-jet speed ratio
        self.blade_jet_speed_ratio = self.u / np.sqrt(2 * 9.81 * self.deltah_ambis)

        # Nitrogen required calculations
        self.T01_nitro = 300  # K
        self.cp_nitro = 1040  # J/kg/K
        self.R_nitro = 296.8  # J/kg/K
        self.gamma_nitro = 1.4

        self.T3_nitro = self.T01_nitro - 0.5 * self.c3**2 / self.cp_nitro
        self.a3_nitro = np.sqrt(self.gamma_nitro * self.R_nitro * self.T3_nitro)
        self.M3_nitro = self.c3 / self.a3_nitro
        self.p3_nitro = 1e5 / (1 + 0.5 * (self.gamma_nitro - 1) * self.M3_nitro**2)**(self.gamma_nitro/(self.gamma_nitro-1))
        self.rho_3_nitro = self.p3_nitro / (self.R_nitro * self.T3_nitro)
        self.mdot = self.doa * self.rho_3_nitro * self.c3m * self.d_mean * self.Height
        self.T_amb_is_nitro = self.T01_nitro * (self.pamb/self.p01)**((self.gamma_nitro - 1)/self.gamma_nitro)
        self.deltah_ambis_nitro = self.cp_nitro * (self.T01_nitro - self.T_amb_is_nitro)
        self.eff_nitro = self.deltah_useful / self.deltah_ambis_nitro


    def pretty_print(self):
        print(f"Inputs:")
        print(f"  {'Power (P)':<30} {self.P/1000:<10.4g} kW")
        print(f"  {'RPM':<30} {self.RPM:<10.4g} RPM")
        print(f"  {'Mean Diameter (d_mean)':<30} {self.d_mean_mm:<10.4g} mm")
        # print(f"  {'GG Temperature (T01)':<30} {self.T01:<10.4g} K")
        # print(f"  {'GG Heat Capacity Ratio (gamma)':<30} {self.gamma:<10.4g}")
        print(f"  {'GG mdot':<30} {self.mdot*1000:<10.4g} g/s")
        # print(f"  {'GG Heat Capacity':<30} {self.cp:<10.4g} J/Kg/K")
        # print(f"  {'GG Gas Constant':<30} {self.R:<10.4g} J/kg/K")
        print(f"  {'GG Chamber Pressure':<30} {self.p01_bar:<10.4g} bar")
        print(f"  {'Nozzle Angle':<30} {np.rad2deg(self.beta):<10.4g} degree")
        print(f"  {'Degree of Admission':<30} {self.doa*100:<10.4g}%")
        print(f"Outputs:")
        print(f"  {'Rotor surface speed (u)':<30} {self.u:<10.4g} m/s")
        print(f"  {'Specific heat delta (dh_use)':<30} {self.deltah_useful:<10.4g} J/kg")
        print(f"  {'Absolute Circ. Velocity (c3u)':<30} {self.c3u:<10.4g} m/s")
        print(f"  {'Absolute Meri. Velocity (c3m)':<30} {self.c3m:<10.4g} m/s")
        print(f"  {'Absolute Velocity (c3)':<30} {self.c3:<10.4g} m/s")
        print(f"  {'Exit Temperature (T3)':<30} {self.T3:<10.4g} K")
        print(f"  {'Exit sonic velocity (a3)':<30} {self.a3:<10.4g} m/s")
        print(f"  {'Exit Pressure (p3)':<30} {self.p3/1e5:<10.4g} bar")
        print(f"  {'Exit Density (rho_3)':<30} {self.rho_3:<10.4g} kg/m3")
        print(f"  {'Required Mach Number (M3)':<30} {self.M3:<10.4g}")
        print(f"  {'Required Blade Height (H_3)':<30} {self.Height*1000:<10.4g} mm")
        print(f"  {'Isen Ambient Temp (t_amb_is)':<30} {self.T_amb_is:<10.4g} K")
        print(f"  {'Turbine Efficiency (eff)':<30} {self.eff:<10.4g}")
        print(f"  {'Blade-Jet Speed Ratio':<30} {self.blade_jet_speed_ratio:<10.4g}")   
        print(f" Nitrogen Required Calculations:")
        print(f"  {'Exit Temperature (T3)':<30 } {self.T3_nitro:<10.4g} K")
        print(f"  {'Exit sonic velocity (a3)':<30} {self.a3_nitro:<10.4g} m/s")
        print(f"  {'Exit Pressure (p3)':<30} {self.p3_nitro/1e5:<10.4g} bar")
        print(f"  {'Exit Density (rho_3)':<30} {self.rho_3_nitro:<10.4g} kg/m3")
        print(f"  {'Required Mach Number (M3)':<30} {self.M3_nitro:<10.4g}")
        print(f"  {'Computed mdot (mdot)':<30} {self.mdot*1000:<10.4g} g/s")
        print(f"  {'Isen Ambient Temp (t_amb_is)':<30} {self.T_amb_is_nitro:<10.4g} K")
        print(f"  {'Turbine Efficiency (eff)':<30} {self.eff_nitro:<10.4g}")    

# Example usage
if __name__ == "__main__":
    gasgen = GasGenerator(ox='N2O', fuel='Isopropanol')
    Tg_c, Cp_c, R_c, gam_c = gasgen.combustion_sim(OF=0.5, pc=20)
    print(f"Combustion Results:")
    print(f"  {'Combustion Temperature (Tc)':<30} {Tg_c:<10.4g} K")
    print(f"  {'Combustion Heat Capacity (Cp)':<30} {Cp_c:<10.4g} J/kg/K")
    print(f"  {'Combustion Gas Constant (R)':<30} {R_c:<10.4g} J/kg/K")
    print(f"  {'Combustion  Gamma (gam)':<30} {gam_c:<10.4g}")  

    stage = Turbine(
        P=13000,       # W
        RPM=20000,     
        d_mean_mm=100, # mm
        T01=Tg_c,       # K
        gamma=gam_c,
        mdot=0.04,     # kg/s
        cp=Cp_c,       # J/kg/K
        R=R_c,         # J/kg/K
        p01_bar=20,    # bar
        beta_deg=15,   # degrees
        doa=0.2,       # degree of admission
        pamb=1e5)     # ambient pressure [Pa]
    stage.pretty_print()
    

    


Combustion Results:
  Combustion Temperature (Tc)    1063       K
  Combustion Heat Capacity (Cp)  1.041e+04  J/kg/K
  Combustion Gas Constant (R)    572.3      J/kg/K
  Combustion  Gamma (gam)        1.137     
Inputs:
  Power (P)                      13         kW
  RPM                            2e+04      RPM
  Mean Diameter (d_mean)         100        mm
  GG mdot                        40         g/s
  GG Chamber Pressure            20         bar
  Nozzle Angle                   15         degree
  Degree of Admission            20        %
Outputs:
  Rotor surface speed (u)        104.7      m/s
  Specific heat delta (dh_use)   3.25e+05   J/kg
  Absolute Circ. Velocity (c3u)  1656       m/s
  Absolute Meri. Velocity (c3m)  443.9      m/s
  Absolute Velocity (c3)         1715       m/s
  Exit Temperature (T3)          922.2      K
  Exit sonic velocity (a3)       774.7      m/s
  Exit Pressure (p3)             1.811      bar
  Exit Density (rho_3)           0.343      kg/m3
  Re

ValueError: Unknown format code '\x20' for object of type 'str'

: 